In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import os
from lips import get_root_path
# from lips.benchmark.airfransBenchmark import AirfRANSBenchmark
from lips.dataset.airfransDataSet import AirfRANSDataSet
import numpy as np

In [ ]:
LIPS_PATH = get_root_path()
DIRECTORY_NAME = 'Dataset'
BENCHMARK_NAME = "DEFAULT"
LOG_PATH = LIPS_PATH + "lips_logs.log"
BENCH_CONFIG_PATH = os.path.join("airfoilConfigurations","benchmarks","confAirfoil.ini") 

directory_name='Dataset'

In [9]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(7, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 4)
        )
    
    def forward(self, x):
        return self.model(x)

In [11]:
def train_model(model, train_loader, val_loader, epochs=50, lr=1e-3):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    device = torch.device("cpu")
    model.to(device)
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                y_pred = model(X_val)
                loss = criterion(y_pred, y_val)
                val_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss/len(train_loader):.6f} | Val Loss: {val_loss/len(val_loader):.6f}")
    
    return model

In [2]:
def load_data(dataset):
    X_attr = ["x-position", "y-position", "x-inlet_velocity", "y-inlet_velocity", "x-normals", "y-normals", "distance_function"]
    y_attr = ["x-velocity", "y-velocity", "pressure", "turbulent_viscosity"]
    X, y = [], []
    
    for i in range(1,10):
        X.append(np.column_stack([dataset.data[attr][i] for attr in X_attr]))
        y.append(np.column_stack([dataset.data[attr][i] for attr in y_attr]))

    X = np.vstack(X)
    y = np.vstack(y)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
    X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)
    
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)
    
    return train_loader, test_loader

In [3]:
attr_names = (
        'x-position',
        'y-position',
        'x-inlet_velocity', 
        'y-inlet_velocity', 
        'distance_function', 
        'x-normals', 
        'y-normals', 
        'x-velocity', 
        'y-velocity', 
        'pressure', 
        'turbulent_viscosity',
    )
attr_x = attr_names[:7]
attr_y = attr_names[7:]

In [4]:
configuration_file = None #Convenient alternative but not required at this point
dataset_name = "my_dataset_test"
usecase_task = "scarce" #Four task are supported: 'full', 'scarce', 'reynolds', 'aoa'
usecase_split = "training" #Describe which data subset within a task to be used, the other option is testing
log_path = "dataset_log"

In [5]:
my_dataset = AirfRANSDataSet(config = configuration_file, 
                             name = dataset_name,
                             task = usecase_task,
                             split = usecase_split,
                             attr_names = attr_names, 
                             attr_x = attr_x, 
                             attr_y = attr_y, 
                             log_path = log_path)

In [6]:
my_dataset.load(path=DIRECTORY_NAME)

Loading dataset (task: scarce, split: train): 100%|██████████| 200/200 [00:36<00:00,  5.54it/s]


In [16]:
my_dataset.data["x-position"]

array([ 4.21555805,  4.21557093,  3.99052548, ..., -2.13636851,
       -2.14823389, -2.16010475])

In [7]:
train_loader, test_loader = load_data(my_dataset)

In [10]:
model = MLP()

In [12]:
trained_model = train_model(model, train_loader, test_loader)

Epoch 1/50 | Train Loss: 328.363312 | Val Loss: 315.561279
Epoch 2/50 | Train Loss: 315.177704 | Val Loss: 302.677582
Epoch 3/50 | Train Loss: 302.291046 | Val Loss: 289.427948
Epoch 4/50 | Train Loss: 289.031219 | Val Loss: 276.001648
Epoch 5/50 | Train Loss: 275.601746 | Val Loss: 261.712982
Epoch 6/50 | Train Loss: 261.306366 | Val Loss: 246.266724
Epoch 7/50 | Train Loss: 245.864716 | Val Loss: 229.595932
Epoch 8/50 | Train Loss: 229.194305 | Val Loss: 211.650818
Epoch 9/50 | Train Loss: 211.252518 | Val Loss: 192.639771
Epoch 10/50 | Train Loss: 192.248947 | Val Loss: 172.640686
Epoch 11/50 | Train Loss: 172.266815 | Val Loss: 151.939362
Epoch 12/50 | Train Loss: 151.590973 | Val Loss: 130.824799
Epoch 13/50 | Train Loss: 130.500977 | Val Loss: 109.563049
Epoch 14/50 | Train Loss: 109.273392 | Val Loss: 88.723915
Epoch 15/50 | Train Loss: 88.482239 | Val Loss: 69.020523
Epoch 16/50 | Train Loss: 68.836746 | Val Loss: 51.185768
Epoch 17/50 | Train Loss: 51.071552 | Val Loss: 35.881